# Implementation of attention all you need paper in python

# New Section

HEllo it is my project **santa** it is* english to tamil* translator .for that i going to use **transformer  ** let bulid from scratch

In [ ]:
from datasets import load_dataset
from huggingface_hub import login
login("enter hf code dude")

now load the dataset from hf,we use NLPC-UOM/English-Tamil-Parallel-Corpus it has 30 k parallal corpus .let analyze the structure  of the data

In [ ]:
ds=load_dataset("NLPC-UOM/English-Tamil-Parallel-Corpus")

En-Ta-English.txt:   0%|          | 0.00/472k [00:00<?, ?B/s]

En-Ta-Tamil.txt:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

glossary_english.txt:   0%|          | 0.00/347k [00:00<?, ?B/s]

glossary_tamil.txt:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62852 [00:00<?, ? examples/s]

In [ ]:
mid=8950
dataset=[]
for i in range(8950):
    dataset.append((ds['train'][i]['text'],ds['train'][mid-1+ i]['text']))


In [ ]:
for i in range(22477):
    dataset.append((ds['train'][start1+i]['text'],ds['train'][mid1+ i]['text']))

In [ ]:
mid=8950
start1=mid*2-2
mid1=22477+8950*2-2


In [ ]:
dataset[-1]

('mention', 'குறிப்பிடு')

In [ ]:
import pandas as pd
df=pd.DataFrame(dataset,columns=['en','ta'])


In [ ]:
l=[0,1,2,mid,mid+1,mid+2,]
df.drop(l,inplace=True,axis=0)

In [ ]:
ds['train'][start1]['text']

'#NLPC En-Ta Glossary prepared by University of Moratuwa'

In [ ]:
df.to_csv("english_tamil.csv")

# i converted it into csv for offline use
**let process the data**

In [ ]:
import pandas as pd
df=pd.read_csv("/content/english_tamil.csv",index_col=0)

In [ ]:
df

,en,ta
3,Ranaviru Sewa Authority,ரணவிரு சேவை அதிகார சபை
4,Annual Report 2011,வருடாந்த அறிக்கை 2011
5,"No.301, 4th Floor,","இல. 301, 4ஆம் மாடி,"
6,T.B.Jayah Mawatha,டி.பி. ஜாயா மாவத்தை
7,Colombo 10,கொழும்பு 10
...,...,...
31422,mental,மன
31423,mental aberration,மனப் பிறழ்ச்சி
31424,mental competency,மனத் தேர்ச்சி
31425,mental deficiency,மன ஊனம்


In [ ]:
def process_enn(len: str ):
  return str(len).strip().lower()
def process_ta(len: str ):
  return str(len).strip()

In [ ]:
process_enn("Annual Report 2011")

'annual report 2011'

In [ ]:
df['en']=df['en'].transform(process_enn)
df['ta']=df['ta'].transform(process_ta)

In [ ]:
import sentencepiece as spm

In [ ]:
df['en'].to_csv("en.txt", index=False, header=False)
df['ta'].to_csv("ta.txt", index=False, header=False)

let bulid tokenizer for our model we need two tokenizer one for tamil,and another for english

for creating tokenizer


In [ ]:
import sentencepiece as spm

  spm.SentencePieceTrainer.Train(
     '--input=/content/en.txt --model_prefix=spm_en --vocab_size=8000'
 )
spm.SentencePieceTrainer.Train(
    '--input=/content/ta.txt --model_prefix=spm_ta --vocab_size=8000 --character_coverage=0.9995 --model_type=unigram'
)


IndentationError: unexpected indent (ipython-input-347564406.py, line 3)

In [23]:
import sentencepiece as spm

# Load the trained model
sp_en = spm.SentencePieceProcessor()
sp_en.load("/content/spm_en.model")
sp_ta = spm.SentencePieceProcessor()
sp_ta.load("/content/spm_ta.model")


True

In [24]:
ta_vocab_size=sp_ta.get_piece_size()
print(ta_vocab_size)

8000


In [25]:
text = "2011 ஆம் வருடத்திற்கான அணுகுமுறை - நிகழ்ச்சித்திட்டத்திற்கான வரவு செலவு 07. ரணவிரு சேவை அதிகார"

def en_encode(text):
  BOS,EOS=1,2
  token_ids = sp_en.encode(text, out_type=int);
  return [BOS]+token_ids+[EOS]


def en_decode(token_ids):
  decoded = sp_en.decode(token_ids[1:-1])
  return decoded


def ta_encode(text):
  BOS,EOS=1,2
  token_ids = sp_ta.encode(text, out_type=int);
  return [BOS]+token_ids+[EOS]


def ta_decode(token_ids):
  decoded = sp_ta.decode(token_ids[1:])
  return decoded


In [26]:

import torch
import torch.nn as nn

pad=0

def process_en(en_txt):
  en_txt=en_encode(en_txt)
  return en_txt

In [ ]:
train=df['en'].apply(process_en)
ytrain=df['ta'].apply(ta_encode)

In [ ]:
train=[torch.tensor(i,dtype=torch.int64) for i in train.to_list()]
ytrain=[torch.tensor(i,dtype=torch.int64) for i in ytrain.to_list()]

In [ ]:
from   torch.nn.utils.rnn import pad_sequence

padded=pad_sequence(train,batch_first=True ,padding_value=pad)
ypadded=pad_sequence(ytrain,batch_first=True ,padding_value=pad)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(padded,ypadded,test_size=1000,random_state=9)

In [ ]:


print(xtrain.size(),ytrain.size())

torch.Size([30421, 133]) torch.Size([30421, 202])


In [ ]:
from torch.utils.data import  DataLoader,Dataset
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        return x, y


dataset_train=MyDataset(xtrain,ytrain)
dataset_test=MyDataset(xtest,ytest)
dataloader_train=DataLoader(dataset_train,batch_size=32,shuffle=True)
dataloader_test=DataLoader(dataset_test,batch_size=32,shuffle=True)


# **Let bulid architecture of the Transformer**

# Encoder


1.    MulitHeadAttention
2.  FeedForward
3.    PositionalEmbedding
2.   ScaleDotProduct







In [2]:


import torch.nn.functional as F
import torch.nn as nn
import torch
class ScaleDotProduct(nn.Module):
  def __init__(self,d_k,drop_rate=0.1):
    super(ScaleDotProduct,self).__init__()

    self.d_k=d_k
    self.drop_rate=drop_rate
    self.dropout=nn.Dropout(drop_rate)

  def forward(self,query,key,value,mask=None):

    score=torch.matmul(query,key.transpose(-1,-2))
    score=score/math.sqrt(self.d_k)
    if mask is not None:
      # Broadcast to match score: [batch_size, heads, seq_len, seq_len]
      score = score.masked_fill(mask, float('-inf'))

    score=F.softmax(score,dim=-1)
    score = torch.nan_to_num(score, nan=0.0)
    score=torch.matmul(self.dropout(score),value)
    return score

In [3]:


class MulitHeadAttention(nn.Module):
  def __init__(self,dmodel,head,drop=0.1):
    super(MulitHeadAttention,self).__init__()
    self.dmodel=dmodel
    self.d_k=dmodel//head
    self.head=head
    self.drop=drop
    self.query=nn.Linear(self.dmodel,self.dmodel)
    self.key=nn.Linear(self.dmodel,self.dmodel)
    self.value=nn.Linear(self.dmodel,self.dmodel)
    self.attention=ScaleDotProduct(self.d_k)
    self.Wo=nn.Linear(self.dmodel,self.dmodel)
    self.dropout=nn.Dropout(drop)

  def group_head(self,x):
    """ note i am only done this Combines multiple heads into a single embedding dimension
    # Input shape: (batch, seq_len, head, d_k)
    # Output shape: (batch, seq_len, head * d_k)"""
    return x.view(x.size(0),x.size(1),self.head*self.d_k)


  def group_split (self,x):
      """ return (batch,seq_len,head,d_k)"""
      return x.view(x.size(0),x.size(1),self.head,self.d_k)

  def forward(self, query, key, value, mask=None):
      Q = self.group_split(self.query(query))   # (B,L,H,d_k)
      K = self.group_split(self.key(key))       # (B,L,H,d_k)
      V = self.group_split(self.value(value))   # (B,L,H,d_k)

      # → (B,H,L,d_k)
      Q = Q.permute(0, 2, 1, 3).contiguous()
      K = K.permute(0, 2, 1, 3).contiguous()
      V = V.permute(0, 2, 1, 3).contiguous()

      # Attention
      out = self.attention(Q, K, V, mask=mask)   # (B,H,L,d_k)

      # (B,L,H,d_k)
      out = out.permute(0, 2, 1, 3).contiguous()

      #  (B,L,d_model)
      out = self.group_head(out)

      return self.Wo(self.dropout(out))



In [4]:

class FeedForward(nn.Module):
  def __init__(self,dmodel,d_ff,drop=0.1):
    super(FeedForward,self).__init__()
    self.dmodel=dmodel
    self.d_ff=d_ff
    self.drop=drop
    self.dropout=nn.Dropout(drop)
    self.layer1=nn.Linear(self.dmodel,self.d_ff)
    self.layer2=nn.Linear(self.d_ff,self.dmodel)



  def forward(self,x):
    x=F.relu(self.dropout(self.layer1(x)))
    x=self.layer2(x)
    return x




In [5]:


import math
class PositionalEmbedding(nn.Module):
  def __init__(self,d_model,max_len=300,drop=0.1):
    super(PositionalEmbedding,self).__init__()
    self.d_model=d_model
    self.maxlen=max_len
    self.dropout=nn.Dropout(drop)

    pos=torch.arange(0,max_len,dtype=torch.float).unsqueeze(1)
    div=torch.exp(torch.arange(0,d_model,2).float()*(-1*math.log(10000))/self.d_model)
    pe=torch.zeros(self.maxlen,self.d_model)
    #torch.Size([256]) torch.Size([512, 1])
    pe[:,0::2]=torch.sin(pos/div)
    pe[:,1::2]=torch.cos(pos/div)
    #self.register_buffer('pe', self.pe.unsqueeze(0))
    #self.pe=self.pe.unsqueeze(0).to("cuda" if torch.cuda.is_available() else  "cpu")
    self.register_buffer("pe", pe.unsqueeze(0))
  def forward(self ,x):

    x=  x + self.pe[:,:x.size(1), :]
    return self.dropout(x)


In [6]:
class Encodelayer(nn.Module):
  def __init__(self,d_model,head,d_ff,dropout_rate=0.1):
    super(Encodelayer,self).__init__()
    self.d_model=d_model
    self.head=head
    self.d_ff=d_ff
    self.MHA=MulitHeadAttention(self.d_model,self.head,drop=dropout_rate)
    self.norm1=nn.LayerNorm(d_model, eps=1e-6)
    self.FF=FeedForward(self.d_model,self.d_ff,dropout_rate)
    self.norm2=nn.LayerNorm(d_model, eps=1e-6)
    self.dropout = nn.Dropout(dropout_rate)


  def forward(self,x,mask=None):

    x1=self.MHA(x,x,x,mask)

    x=self.norm1(self.dropout(x1)+x)

    x1=self.FF(x)

    x=self.norm2(self.dropout(x1)+x)

    return x




In [7]:

class Encoder(nn.Module):
  def __init__(self,vocab_size,d_model,d_ff,head,drop_rate=0.15,max_len=5000):
    super(Encoder,self).__init__()
    self.vocab_size=vocab_size
    self.d_model=d_model
    self.d_ff=d_ff
    self.head=head
    self.embedding=nn.Embedding(self.vocab_size,self.d_model,padding_idx=0)
    self.position=PositionalEmbedding(self.d_model,max_len)
    self.layers= nn.ModuleList([Encodelayer(self.d_model,self.head,self.d_ff,dropout_rate=0.1) for _ in range(6)])
    self.norm=nn.LayerNorm(self.d_model)
  def forward(self,x,mask=None):
    x1=self.embedding(x)* math.sqrt(self.d_model)
    x=self.position(x1)
    for i in range(6):
      x=self.layers[i](x,mask)
    x=self.norm(x)
    return x

Now let bulid **Decoder**

In [8]:
class DecoderLayer(nn.Module):
  def __init__(self,dmodel,head,d_ff,droprate=0.1):
    super(DecoderLayer,self).__init__()
    self.atten_layer=MulitHeadAttention(dmodel,head,droprate)
    self.norm1=nn.LayerNorm(dmodel)
    self.en_atten_layer=MulitHeadAttention(dmodel,head,droprate)
    self.norm2=nn.LayerNorm(dmodel)
    self.ff=FeedForward(dmodel,d_ff,droprate)
    self.norm3=nn.LayerNorm(dmodel)
    self.dropout=nn.Dropout(droprate)

  def forward(self,x,en_out,src_mask,tgt_mask):
    x1=self.atten_layer(x,x,x,mask=tgt_mask)
    x=self.norm1(self.dropout(x1)+x)
    x1=self.en_atten_layer(x,en_out,en_out,mask=src_mask)
    x=self.norm2(self.dropout(x1)+x)
    x1=self.ff(x)
    x=self.norm3(self.dropout(x1)+x)
    return x

In [9]:




class Decoder(nn.Module):
  def __init__(self,dmodel,head,d_ff,vocab_size,droprate=0.15):
    super(Decoder,self).__init__()
    self.dmodel=dmodel
    self.head=head
    self.vocab_size=vocab_size
    self.droprate=droprate
    self.embedding=nn.Embedding(self.vocab_size,self.dmodel,padding_idx=0)
    self.position=PositionalEmbedding(self.dmodel)
    self.layers=nn.ModuleList([DecoderLayer(self.dmodel,self.head,d_ff,droprate) for i in range(6)])
    self.norm=nn.LayerNorm(self.dmodel)
    self.dropout=nn.Dropout(droprate)



  def forward(self,x,ec_out,src_mask,tgt_mask):
    x=self.embedding(x)*math.sqrt(self.dmodel)
    x=self.position(x)

    for i in range(6):
      x=self.layers[i](x,ec_out,src_mask,tgt_mask)

    x=self.norm(x)
    return x



In [10]:
class Generator(nn.Module):
  def __init__(self,dmodel,vocab):
    super(Generator,self).__init__()
    self.layer=nn.Linear(dmodel,vocab)

  def forward(self,x):
    return self.layer(x)


In [11]:

import numpy as np
class Transformer(nn.Module):
  def __init__(self,dmodel,head,d_ff,vocab_size,device,droprate):
    super(Transformer,self).__init__()
    self.pad_idx = 0
    self.device=device
    self.encoder=Encoder(d_model=dmodel,head=head,vocab_size=vocab_size,d_ff=d_ff,drop_rate=droprate).to(device)
    self.decoder=Decoder(dmodel=dmodel,head=head,vocab_size=vocab_size,droprate=droprate,d_ff=d_ff).to(device)
    self.generator=Generator(dmodel=dmodel,vocab=vocab_size).to(device)

  def get_pad_mask(self, seq, pad_idx):
    # seq: (B, T)
    # True = pad → mask out
    return (seq == pad_idx).unsqueeze(1).unsqueeze(2)

  def get_subsequent_mask(self, seq):
    # seq: (B, T)
    L = seq.size(1)
    subsequent = torch.triu(
        torch.ones(L, L, device=seq.device), diagonal=1
    ).bool()   # upper triangular → future = True (mask)

    # shape → (1, 1, T, T)
    return subsequent.unsqueeze(0).unsqueeze(1)

  def forward(self,src,tgt):
    if src.dim() == 1:
        src = src.unsqueeze(0)
    if tgt.dim() == 1:
        tgt = tgt.unsqueeze(0)

    src=src.to(self.device)
    tgt=tgt.to(self.device)
    src_mask=self.get_pad_mask(src,self.pad_idx)
    tgt_mask=self.get_pad_mask(tgt,self.pad_idx) | self.get_subsequent_mask(tgt)


    encoder_out=self.encoder(src,src_mask)
    decoder_out=self.decoder(tgt,encoder_out,src_mask,tgt_mask)
    gen=self.generator(decoder_out)
    return gen

In [12]:
model=Transformer(dmodel=512,head=8,d_ff=2048,vocab_size=8000,device='cuda' if torch.cuda.is_available() else 'cpu',droprate=0.15)

In [ ]:
class Noamoptim:
  def __init__(self,optimizer,model_size,warm_up,steps):
    self.optimizer=optimizer
    self.model_size=model_size
    self.warm_up=warm_up
    self.steps=steps

  def zero_grad(self):
    self.optimizer.zero_grad()

  def get_lr(self):
    return self.model_size**(-0.5)*min([self.steps**(-0.5),self.steps*self.warm_up**(-1.5)])

  def step(self):
    self.steps+=1
    lr=self.get_lr()
    for p in self.optimizer.param_groups:
      p['lr']=lr
    self.optimizer.step()

In [20]:
def greedy_decode(model, src, max_len=50, sos_id=1, eos_id=2):
    model.eval()

    # Make batch dimension if needed
    if src.dim() == 1:
        src = src.unsqueeze(0)

    # Start target with <SOS>
    tgt = torch.tensor([[sos_id]], device=src.device)

    for _ in range(max_len):
        # Forward pass
        out = model(src, tgt)             # (B, tgt_len, vocab)

        # Get last token prediction
        next_token = torch.argmax(out[:, -1, :], dim=-1)   # (B,)

        # Append
        tgt = torch.cat([tgt, next_token.unsqueeze(1)], dim=1)

        # Stop at <EOS>
        if next_token.item() == eos_id:
            break

    return tgt[0]   # return sequence


In [62]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim

pad_token_id = 0
criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)
optimizer = optim.Adam(model.parameters(),   lr=0,
    betas=(0.9, 0.98),
    eps=1e-9)
optimizer1=Noamoptim(optimizer=optimizer,model_size=512,steps=0,warm_up=10000)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
st=torch.load('/content/drive/MyDrive/transformer_model_15.pt',map_location=torch.device('cpu'))
model.load_state_dict(st)

<All keys matched successfully>

In [61]:

num=20
for epoch in range(1,num+1):
    model.train()
    total_loss = 0

    progress_bar = tqdm(dataloader_train, desc=f"Epoch {epoch}", leave=False)

    for batch_idx, (src, tgt) in enumerate(progress_bar):
        src = src.to('cuda' if torch.cuda.is_available() else 'cpu')
        tgt = tgt.to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer1.zero_grad()

        output = model(src, tgt[:,:-1])

        loss = criterion(output.view(-1, output.size(-1)), tgt[:,1:].reshape(-1))

        loss.backward()
        optimizer1.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'/content/drive/MyDrive/transformer_model_{epoch}.pt')

    progress_bar1 = tqdm(dataloader_test, desc=f"Epoch {epoch}", leave=False)
    test_loss = 0
    for batch_idx, (src, tgt) in enumerate(progress_bar1):

        src = src.to('cuda' if torch.cuda.is_available() else 'cpu')
        tgt = tgt.to('cuda' if torch.cuda.is_available() else 'cpu')
        model.eval()
        with torch.no_grad():

          output = model(src, tgt[:,:-1])
          # T = tgt.size(0) - 1  # exclude <sos>
          # #output = output[:T]
          loss = criterion(output.view(-1, output.size(-1)), tgt[:,1:].reshape(-1))

        # loss.backward()
        # optimizer1.step()

        test_loss += loss.item()
        progress_bar1.set_postfix(loss=loss.item())

    print(f"Epoch {epoch} completed. Avg Loss: {total_loss / len(dataloader_train):.4f} Test.Loss: {test_loss / len(dataloader_test):.4f}")


In [ ]:
torch.save(model.state_dict(), f'/content/drive/MyDrive/transformer_model_{epoch+1}.pt')

In [ ]:

"""test:Epoch  Avg Loss: 4.5614
train
Epoch 1 completed. Avg Loss: 7.0829
Epoch 2 completed. Avg Loss: 6.0390
Epoch 3 completed. Avg Loss: 5.3813
Epoch 4 completed. Avg Loss: 4.7824
Epoch 5 completed. Avg Loss: 4.2355
Epoch 6 completed. Avg Loss: 3.7630
Epoch 7 completed. Avg Loss: 3.3463
Epoch 8 completed. Avg Loss: 3.0052
Epoch 9 completed. Avg Loss: 2.7386
Epoch 10 completed. Avg Loss: 2.5368
Epoch 11 completed. Avg Loss: 2.556
Epoch 11 completed. Avg Loss: 2.5567
Epoch 12 completed. Avg Loss: 2.2327
Epoch 13 completed. Avg Loss: 1.9432
Epoch 14 completed. Avg Loss: 1.7074"""

Epoch 1 completed. Avg Loss: 4.5902


'test:Epoch 11 completed. Avg Loss: 4.5614\ntrain\nEpoch 1 completed. Avg Loss: 7.0829\nEpoch 2 completed. Avg Loss: 6.0390\nEpoch 3 completed. Avg Loss: 5.3813\nEpoch 4 completed. Avg Loss: 4.7824\nEpoch 5 completed. Avg Loss: 4.2355\nEpoch 6 completed. Avg Loss: 3.7630\nEpoch 7 completed. Avg Loss: 3.3463\nEpoch 8 completed. Avg Loss: 3.0052\nEpoch 9 completed. Avg Loss: 2.7386\nEpoch 10 completed. Avg Loss: 2.5368\nEpoch 11 completed. Avg Loss: 2.556\nEpoch 11 completed. Avg Loss: 2.5567\nEpoch 12 completed. Avg Loss: 2.2327\nEpoch 13 completed. Avg Loss: 1.9432\nEpoch 14 completed. Avg Loss: 1.7074'

'/content/drive/MyDrive/transformer_model_1.pt' ||||

each epoch has 951 steps

let evaluate the model

In [60]:
text=input("enter the letter-").lower()
src=torch.tensor(process_en(text))
text=greedy_decode(model,src)
print(text)
print(ta_decode(text.tolist()))


enter the letter-he is a student
tensor([   1, 1137,  131,  242,   27,    5,   74,    2])
மாணவர் தகைய்வு
